In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/netflix-shows/netflix_titles.csv")
data.head()

# which films are made more



In [ ]:
rating = pd.DataFrame(data.groupby('rating').show_id.count()).reset_index()
rating

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(20,6))
sns.barplot(x="rating", y="show_id", data=rating)

answer: 'TV-MA' --- 17+ or with parents

In [ ]:
# consider which countries are making films
import re

country = pd.DataFrame(data.country.value_counts()).reset_index()
country_unique = set()
for val in list(country['index']):
    country = re.findall(r"[\&\w\s\-]+", val)
    for unique in country:
        country_unique.add(unique.lstrip())

country_cinema = pd.DataFrame()
country_cinema['country'] = list(country_unique)
country_cinema["count"] = [0 for i in country_cinema['country']]
country_cinema

In [ ]:
country_cinema[country_cinema.country == "Armenia"].index[0]

In [ ]:
for country in country_cinema.country:
    i = country_cinema[country_cinema.country == country].index[0]
    for val in data.country:
        val = str(val)
        if val.count(country) > 0:
            country_cinema["count"][i] += 1
            
country_cinema        

In [ ]:
# most of all movies were shot
print(country_cinema[country_cinema["count"] == country_cinema["count"].max()])

In [ ]:
plt.figure(figsize=(60,12))
sns.barplot(x=country_cinema["country"], y=country_cinema["count"])

# what percentage no data?

In [ ]:
data_nan = pd.DataFrame(data.isnull().sum()).reset_index()
data_nan = data_nan.rename(columns={"index":"col", 0:"count_nan"})
data_nan["%"] = data_nan.count_nan / data.shape[0] * 100 
data_nan

What more Movie or TV Show?

In [ ]:
count_type = pd.DataFrame(data.groupby('type').show_id.count()).reset_index()
count_type

In [ ]:
sns.barplot(x=count_type.type, y=count_type.show_id)

In [ ]:
year = pd.DataFrame(data.groupby(['type','release_year']).type.count())
year = year.rename(columns={'type':'count'}).reset_index()
year

In [ ]:
plt.figure(figsize=(20,6))

sns.lineplot(x="release_year", y="count", hue="type", data=year)

Type data:

# most popular actor

In [ ]:
cast = pd.DataFrame(data.cast.value_counts()).reset_index()
cast

In [ ]:
list(data.cast.unique())

In [ ]:
actors_unique = set()
for val in list(cast['index']):
    actors = val.split(",")
    for unique in actors:
        actors_unique.add(unique.lstrip())

data_actors = pd.DataFrame()
data_actors['actor'] = list(actors_unique)
data_actors["count_cinema"] = [0 for i in data_actors['actor']]
data_actors

In [ ]:
for actor in data_actors.actor:
    i = data_actors[data_actors.actor == actor].index[0]
    for vals in data.cast:
        vals = str(vals).split(",")
        for val in vals:
            if val == actor:
                data_actors.count_cinema[i] += 1
            
data_actors 

In [ ]:
name = data_actors[data_actors.count_cinema == data_actors.count_cinema.max()].actor.values[0]
count_cinema = data_actors[data_actors.count_cinema == data_actors.count_cinema.max()].count_cinema.values[0]

In [ ]:
# most popular actor is ....
print("most popular actor is ", name, " in ", count_cinema, " cinema")

In [ ]:
data.dtypes

Percent values = nan in rating 

# I don’t know yet why calculations at the bottom

In [ ]:
def Type_cinema(column, name_new_columns):
    cinema = []
    
    if name_new_columns == "Movie":
        i = [1,0]
    else:
        i = [0,1]
        
    for val in column:
        if val == "Movie":
            cinema.append(i[0])
        else :
            cinema.append(i[1])
    return cinema

In [ ]:
X = pd.DataFrame()
X["show_id"] = data.show_id
X["Movie"] = Type_cinema(data.type, "Movie")
X["TV_Show"] = Type_cinema(data.type, "TV_Show")
X

In [ ]:
# break the release date into intervals

min_year = data.release_year.min()
max_year = data.release_year.max()
n = data.release_year.count()
print(min_year, max_year, n)

import math

# to determine the size of the interval, we use the Sturges formula

h = round((max_year - min_year) / (1 + math.log(n,2))) 

print(h)

# we divide the initial data into m intervals

m = round(1 + math.log(n,2))

print(m)

# initial value

x_start = min_year - math.floor(h/2)

print(x_start)

# we get intervals

intervals = []
for i in range(m):
    
    interval = [x_start, x_start + h]
    intervals.append(interval)
    x_start = interval[1]

print(intervals)

In [ ]:
i = 0
for interval in intervals:
    i = i + 1
    value = []
    if i == m:
        for value_col in data.release_year:
            if value_col <= int(interval[1]) and value_col >= int(interval[0]):
                val = 1
            else:
                val = 0
            value.append(val)
    else:
        for value_col in data.release_year:
            if value_col < int(interval[1]) and value_col >= int(interval[0]):
                val = 1
            else:
                val = 0
            value.append(val)
    
    X["["+str(interval[0])+" ,"+str(interval[1]) + ")"] = value

In [ ]:
X

In [ ]:
# whether we split the data into a series or not

serials = []
for val in data.duration:
    if val.count("Season") > 0:
        serials.append(1)
    else:
        serials.append(0)

In [ ]:
X["Serial"] = serials

In [ ]:
X

In [ ]:
# add time
pattern = r"[\d]+"

time_min = []
for val in data.duration:
    if val.count("min") > 0:
        time_min.append(re.findall(pattern, val)[0])
    else:
        time_min.append(None)

In [ ]:
X["time"] = time_min

In [ ]:
X

In [ ]:
X.dtypes

In [ ]:
# let's try to figure out the genre

listed_in_unique = set()
for val in list(data.listed_in.unique()):
    listed = re.findall(r"[\&\w\s\-]+\'[\&\w\s\-]+|[\&\w\s\-]+", val)
    for unique in listed:
        listed_in_unique.add(unique.lstrip())
listed_in_unique


In [ ]:
for genre in listed_in_unique:
    a = []
    for val in data.listed_in:
        if val.count(genre) > 0:
            a.append(1)
        else:
            a.append(0)
    X[genre] = a 

In [ ]:
X

# Reting cinema 

* 'TV-PG' --- 0+ but parents
* 'TV-MA' --- 17+ or with parents
* 'TV-Y7-FV' --- 7+
* 'TV-Y7' --- 7+
* 'TV-14' --- 13+
* 'R' --- 17+ or with parents
* 'TV-Y' --- 0+
* 'NR' --- not rated
* 'PG-13' --- 13+
* 'TV-G' --- 0+
* 'PG' (Parental quidance) --- 0+ but parents
* 'G' (General) --- 0+
* 'UR' (unrated) --- before rating introduction
* 'NC-17' --- 17+

In [ ]:
# Reting cinema 
data.rating.unique()

In [ ]:
# parents will not be considered
ratings = {"0+":['TV-PG', 'TV-Y', 'TV-G', 'PG', 'G' ], 
           "7+":['TV-Y7-FV', 'TV-Y7'], 
           "13+":['TV-14', 'PG-13'], 
           "17+":['NC-17', 'TV-MA', 'R'],
           "not_rated":['NR', None, 'UR']}


for category in ratings:
    
    a = []
    for val in data.rating:
        i = 0
        for rating in ratings[category]:
            if val == rating:
                i = 1
        if i == 1:
            a.append(1)
        else :
            a.append(0)
    
    X[category] = a

In [ ]:
X.head()

In [ ]:
X.shape[0] == data.shape[0]

In [ ]:
data_nan = pd.DataFrame(X.isnull().sum()).reset_index()
data_nan = data_nan.rename(columns={"index":"col", 0:"count_nan"})
data_nan["%"] = data_nan.count_nan / data.shape[0] * 100 
data_nan

In [ ]:
show = data.groupby('country').show_id.count()
show[show ==

In [ ]:
data[data.title == "Love"]